## Imports

In [1]:
import pandas as pd
from gurobipy import *

## Helper functions

In [2]:
def readData():
    xls = pd.ExcelFile("ourdata.xlsx")
    
    # Getting point coordinates
    sheet = xls.parse(0) 
    
    xCoordinates = list(sheet['X-coordinate'])
    yCoordinates = list(sheet['Y-coordinate'])
    points = [(xCoordinates[i], yCoordinates[i]) for i in range(len(xCoordinates))]
    
    # Getting block data
    sheet = xls.parse(1, skiprows = [0])
    
    xCoordinates = list(sheet['X coordinate'])
    yCoordinates = list(sheet['Y coordinate'])
    widths = list(sheet['Width'])
    heights = list(sheet['Height'])
    blocks = [(xCoordinates[i], yCoordinates[i], widths[i], heights[i]) for i in range(len(xCoordinates))]
    
    return points, blocks

In [3]:
def manhattanDistance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    
    return abs(x1 - x2) + abs(y1 - y2)

In [4]:
def calculateDistances(points):
    Cij = [[1] * len(points) for i in range(len(points))]
    
    for i in range(0, len(points)):
        for j in range(0, len(points)):
            Cij[i][j] = manhattanDistance(points[i], points[j])

    return Cij

In [5]:
def calculateBlocks(points, blocks):
    Bij = [[1] * len(points) for i in range(len(points))]
    
    for i in range(len(points)):
        for j in range(len(points)):
            for block in blocks:
                count = checkBlockBetweenTwoPoints(points[i], points[j], block)
                if count == 0:
                    Bij[i][j] = 0
            
    return Bij

In [6]:
def checkBlockBetweenTwoPoints(point1, point2, block):
    p1_x, p1_y = point1
    p2_x, p2_y = point2
    b_x, b_y, b_w, b_h = block
    
    # X direction
    dir_x = 1 if p1_x < p2_x else -1
    
    # Y direction
    dir_y = 1 if p1_y < p2_y else -1
    
    count = 0 
    
    # 1st - main path
    checkpoint_1 = True
    for i in range(p1_x, p2_x, dir_x):
        if blockContainsPoint(block, (i, p1_y)) is True:
            checkpoint_1 = False
            break
    
    if checkpoint_1 is True:
        checkpoint_2 = True
        for i in range(p1_y, p2_y, dir_y):
            if blockContainsPoint(block, (p2_x, i)) is True:
                checkpoint_2 = False
                break
        if checkpoint_2 is True:
            count += 1

    # 2nd - main path
    checkpoint_3 = True
    for i in range(p1_y, p2_y, dir_y):
        if blockContainsPoint(block, (p1_x, i)) is True:
            checkpoint_3 = False
            break
    
    if checkpoint_3 is True:
        checkpoint_4 = True
        for i in range(p1_x, p2_x, dir_x):
            if blockContainsPoint(block, (i, p2_y)) is True:
                checkpoint_4 = False
                break
        if checkpoint_4 is True:
            count += 1
        
    return count

In [7]:
def blockContainsPoint(block, point):
    p_x, p_y = point
    b_x, b_y, b_w, b_h = block
    
    return (b_x <= p_x <= b_x + b_w) and (b_y <= p_y <= b_y + b_h)

## Parameters & Constants

In [8]:
points, blocks = readData()
N = len(points)
C = calculateDistances(points)
B = calculateBlocks(points, blocks)

## Creating Model

In [9]:
model = Model()

Using license file /Users/fuadaghazada/gurobi.lic
Academic license - for non-commercial use only


## Decision Variables

In [10]:
x = {}
for i in range(N):
    for j in range(N):
        x[i, j] = model.addVar(vtype = GRB.BINARY, name = f"X_{i}_{j}")
    model.update()
    
u = {}
for i in range(N):
    u[i] = model.addVar(vtype = GRB.CONTINUOUS, name = f"U_{i}")

## Constraints

In [11]:
for i in range(N):    
    model.addConstr(sum(x[i, j] for j in range(N)), GRB.EQUAL, 1)
    
model.update()

In [12]:
for j in range(N):
    model.addConstr(sum(x[i, j] for i in range(N)), GRB.EQUAL, 1)
    
model.update()

In [13]:
for i in range(N):
    model.addConstr(x[i, i], GRB.EQUAL, 0)
    
model.update()

In [14]:
for i in range(1, N):
    for j in range(1, N):
        if i != j:
            model.addConstr(u[i] - u[j] + (N - 1) * x[i, j], GRB.LESS_EQUAL, N - 2)

model.update()

In [15]:
for i in range(N):
    for j in range(N):
        model.addConstr(x[i, j], GRB.LESS_EQUAL, B[i][j])
        
model.update()

## Objective function

In [16]:
obj = 0
for i in range(N):
    for j in range(N):
        obj += C[i][j] * x[i, j] 

In [17]:
model.update()
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 5002 rows, 2550 columns and 14606 nonzeros
Model fingerprint: 0xa2dddca9
Variable types: 50 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 2550 rows and 189 columns
Presolve time: 0.02s
Presolved: 2452 rows, 2361 columns, 11552 nonzeros
Variable types: 49 continuous, 2312 integer (2312 binary)

Root relaxation: objective 3.025306e+02, 176 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  302.53061    0   63          -  302.53061      -     -    0s
     0     0  362.04082    0  102          -  362.04082      -     -    0s
     0     0  363.00000    0   98          -  363.00000      -     -    0s
     0     

In [18]:
for v in model.getVars():
    if v.X != 0:
        print("%s %f" % (v.Varname, v.X))

X_0_13 1.000000
X_1_22 1.000000
X_2_32 1.000000
X_3_37 1.000000
X_4_46 1.000000
X_5_9 1.000000
X_6_42 1.000000
X_7_8 1.000000
X_8_1 1.000000
X_9_43 1.000000
X_10_11 1.000000
X_11_18 1.000000
X_12_39 1.000000
X_13_34 1.000000
X_14_30 1.000000
X_15_40 1.000000
X_16_24 1.000000
X_17_0 1.000000
X_18_3 1.000000
X_19_49 1.000000
X_20_5 1.000000
X_21_10 1.000000
X_22_16 1.000000
X_23_12 1.000000
X_24_35 1.000000
X_25_17 1.000000
X_26_48 1.000000
X_27_4 1.000000
X_28_14 1.000000
X_29_41 1.000000
X_30_20 1.000000
X_31_44 1.000000
X_32_38 1.000000
X_33_31 1.000000
X_34_21 1.000000
X_35_23 1.000000
X_36_15 1.000000
X_37_7 1.000000
X_38_33 1.000000
X_39_28 1.000000
X_40_6 1.000000
X_41_47 1.000000
X_42_29 1.000000
X_43_2 1.000000
X_44_45 1.000000
X_45_26 1.000000
X_46_19 1.000000
X_47_27 1.000000
X_48_36 1.000000
X_49_25 1.000000
U_1 10.000000
U_2 25.000000
U_3 6.000000
U_4 43.000000
U_5 22.000000
U_6 37.000000
U_7 8.000000
U_8 9.000000
U_9 23.000000
U_10 3.000000
U_11 4.000000
U_12 16.000000
U_14